In [1]:
import numpy as np
import os
import pandas as pd
import urllib.request

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils.np_utils import to_categorical
from zipfile import ZipFile

Using TensorFlow backend.


In [2]:
# I like to declare constants to avoid typos later on especially if they'd be used in more than one place
ReluActivation = 'relu'
SoftmaxActivation = 'softmax'
AdamOptimizer = 'adam'
CategoricalCrossEntropy = 'categorical_crossentropy'

In [3]:
def fetch_data_sets():
    path = './data'
    print('fetching test and train datasets from kaggle . . .')
    url = 'https://www.kaggle.com/c/3004/download-all'
    train_set_csv_path = os.path.join(path, 'train.csv')
    test_set_csv_path = os.path.join(path, 'test.csv')
    zipped_file_path = os.path.join(path, 'data.zip')
    if os.path.exists(train_set_csv_path) and os.path.exists(test_set_csv_path):
        return
    os.makedirs(path, exist_ok=True)
    urllib.request.urlretrieve(url, zipped_file_path)
    zf = ZipFile(zipped_file_path, 'r')
    zf.extractall(path=path)
    zf.close()

In [4]:
fetch_data_sets()

fetching test and train datasets from kaggle . . .


In [5]:
training_set = pd.read_csv('./data/train.csv')
test_set = pd.read_csv('./data/test.csv')

In [6]:
Y_train = training_set[['label']]

X_train = training_set.drop(training_set.columns[[0]], axis=1)
X_test = test_set

X_train = np.array(X_train)
X_test = np.array(X_test)

# Reshape the training and test set
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

# Padding the images by 2 pixels since in the paper input images were 32x32
X_train = np.pad(X_train, ((0, 0), (2, 2), (2, 2), (0, 0)), 'constant')
X_test = np.pad(X_test, ((0, 0), (2, 2), (2, 2), (0, 0)), 'constant')

# Standardization
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)
X_train = (X_train - mean_px) / std_px

Y_train = to_categorical(Y_train)

In [7]:
# Building the model
model = Sequential()
model.add(Conv2D(filters=6, kernel_size=5, strides=1, activation=ReluActivation,
                 input_shape=(32, 32, 1)))  # The original paper used sinh/tanh activation
model.add(MaxPooling2D(pool_size=2, strides=2))  # The original paper used Mean/Average pooling
model.add(Conv2D(filters=16, kernel_size=5, strides=1, activation=ReluActivation, input_shape=(14, 14, 6)))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Flatten())

# Now, the fully connected layers
model.add(Dense(units=120, activation=ReluActivation))
model.add(Dense(units=84, activation=ReluActivation))

# Adding the softmax which did not originally exist in the paper
model.add(Dense(units=10, activation=SoftmaxActivation))

In [ ]:
# compile and fit
model.compile(optimizer=AdamOptimizer, loss=CategoricalCrossEntropy, metrics=['accuracy'])

model.fit(X_train, Y_train, steps_per_epoch=10, epochs=42)

Epoch 1/42
10/10 [==============================] - 57s 6s/step - loss: 1.9902 - accuracy: 0.4265
Epoch 2/42
10/10 [==============================] - 62s 6s/step - loss: 1.0294 - accuracy: 0.7319
Epoch 3/42
10/10 [==============================] - 51s 5s/step - loss: 0.4433 - accuracy: 0.8697
Epoch 4/42
10/10 [==============================] - 50s 5s/step - loss: 0.2921 - accuracy: 0.9122
Epoch 5/42
10/10 [==============================] - 55s 6s/step - loss: 0.2177 - accuracy: 0.9355
Epoch 6/42
10/10 [==============================] - 54s 5s/step - loss: 0.1676 - accuracy: 0.9504
Epoch 7/42
10/10 [==============================] - 53s 5s/step - loss: 0.1319 - accuracy: 0.9611
Epoch 8/42
10/10 [==============================] - 55s 6s/step - loss: 0.1072 - accuracy: 0.9684
Epoch 9/42
10/10 [==============================] - 49s 5s/step - loss: 0.0902 - accuracy: 0.9740
Epoch 10/42
10/10 [==============================] - 48s 5s/step - loss: 0.0780 - accuracy: 0.9775
Epoch 11/42
 5/10 [

In [ ]:
# make prediction usin the model we just trained
y_pred = model.predict(X_test)

labels = np.argmax(y_pred, axis=1)

index = np.arange(1, 28001)

labels = labels.reshape([len(labels), 1])
index = index.reshape([len(index), 1])

In [ ]:
# save prediction to file
solution_data_frame = pd.DataFrame()
solution_data_frame['ImageId'] = index
solution_data_frame['Label'] = labels

solution_data_frame.to_csv('./data/solution.csv')